In [1]:
# !pip install torch --quiet
# !pip uninstall torchtext -y
# !pip install torchtext==0.10.0 --quiet

Found existing installation: torchtext 0.14.1
Uninstalling torchtext-0.14.1:
  Successfully uninstalled torchtext-0.14.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 1.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.9.0 which is incompatible.


In [1]:
from torchtext.datasets import IMDB as YelpReviewFull

In [2]:
# !pip install torchdata  # restart after installation
train_iter = YelpReviewFull(split='train')

aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:05<00:00, 15.4MB/s]


In [3]:
next(train_iter)

('neg',
 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [4]:
for (line_number, (label, line)) in enumerate(train_iter):
  print(label, line)
  if line_number == 19:
    break 

neg "I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn't true. I've seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don't exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we're treated to the site of Vincent Gallo's throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, and the same 

In [5]:
from torch.utils.data import DataLoader
train_iter = YelpReviewFull(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False)

In [6]:
next(iter(dataloader))

[('neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg'),
 ('I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. 

In [7]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [8]:
tokenizer = get_tokenizer("basic_english")
train_iter = YelpReviewFull(split='train')

In [9]:
def yield_tokens(data_iter):
  for _, text in data_iter:
    yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])

In [10]:
vocab.set_default_index(vocab["<unk>"])

In [ ]:
# vocab(['here', 'is', 'an', 'alien', 'invasion', 'bangalore'])
vocab["<unk>"]

0

In [12]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: 0 if x == 'neg' else 1

text_pipeline("I am sid")
label_pipeline("neg")

0

In [13]:
from torch.utils.data import DataLoader
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# def collate_fn(batch):
#   src_batch, tgt_batch = [], []
#   for src_batch, tgt_batch in batch:
#     src_batch.append(text_transform[SRC_language](src_sample.rstrip('\n')))
#     tgt_batch.append(text_transform[SRC_language](src_sample.rstrip('\n')))
#   src_batch = pad_sequences(src_batch, padding_value=PAD_IDX)
#   tgt_batch = pad_sequences(tgt_batch, padding_value=PAD_IDX)
#   return src_batch, tgt_batch

In [14]:
weight = torch.randn(3,4)
weight

tensor([[ 0.1527,  0.4260,  0.1989,  0.4360],
        [ 0.7944, -1.0402,  0.9145,  0.0276],
        [-0.4746,  0.0200,  0.0773,  0.3196]])

In [15]:
indices = torch.tensor([2,1])
indices

tensor([2, 1])

In [16]:
embedding = torch.nn.functional.embedding(indices, weight)
embedding

tensor([[-0.4746,  0.0200,  0.0773,  0.3196],
        [ 0.7944, -1.0402,  0.9145,  0.0276]])

In [17]:
embedding.mean(dim=0, keepdim=True)

tensor([[ 0.1599, -0.5101,  0.4959,  0.1736]])

In [18]:
torch.nn.functional.embedding_bag(indices, weight, torch.tensor([0]), mode='mean')

tensor([[ 0.1599, -0.5101,  0.4959,  0.1736]])

In [19]:
import sys
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    # print(((batch[0][1])))  # len(batch) = 64 i.e 1 batch is a list containing 64 elements. batch[0] is of length 2 with the first one containing label and second containing sentence
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        #  print(processed_text)
        #  print(label_list)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
        #  print(offsets)
        #  print(text_list)
        #  sys.exit()
    label_list = torch.tensor(label_list, dtype=torch.int64)  
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    # print(len(text_list))
    text_list = torch.cat(text_list)  # make a long long list containing numeric rep of all sentences one after another.
    # print(len(text_list))
    # sys.exit()
    return label_list.to(device), text_list.to(device), offsets.to(device)    

In [20]:
train_iter = YelpReviewFull(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [21]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        # torch.nn.functional.embedding_bag(indices, weight, torch.tensor([0]), mode="mean")
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)  # making a custom embedding with embedding of dim as "embed_dim"
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [22]:
train_iter = YelpReviewFull(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [26]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1) # disuccees
        optimizer.step()
        # print("predicted label and label is ", predited_label, label)
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        # print("total_count and total_acc is ", total_count, total_acc )
        # sys.exit()
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [27]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = YelpReviewFull()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |    50/  372 batches | accuracy    0.827
| epoch   1 |   100/  372 batches | accuracy    0.842
| epoch   1 |   150/  372 batches | accuracy    0.853
| epoch   1 |   200/  372 batches | accuracy    0.848
| epoch   1 |   250/  372 batches | accuracy    0.848
| epoch   1 |   300/  372 batches | accuracy    0.840
| epoch   1 |   350/  372 batches | accuracy    0.850
-----------------------------------------------------------
| end of epoch   1 | time: 14.98s | valid accuracy    0.843 
-----------------------------------------------------------
| epoch   2 |    50/  372 batches | accuracy    0.852
| epoch   2 |   100/  372 batches | accuracy    0.852
| epoch   2 |   150/  372 batches | accuracy    0.843
| epoch   2 |   200/  372 batches | accuracy    0.847
| epoch   2 |   250/  372 batches | accuracy    0.857
| epoch   2 |   300/  372 batches | accuracy    0.850
| epoch   2 |   350/  372 batches | accuracy    0.864
-----------------------------------------------------------
| e

In [28]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.873
